# Tensorflow 2 SSD Mobilenet Model

# Prerequisites

1. In a terminal session on this Jupyter notebook server, run `aws configure`. This allows this notebook server to access Panorama resources and deploy applications on your behalf.

2. **VERY IMPORTANT** : This example uses a batch of 8, this means we have to use ATLEAST 2 CAMERAS for this example. Batch size 8 is suitable for Jetson Xavier AGX. And for devices using Jetson Xavier NX module, please select batch size 6 instead of 8. please refer to this [link](https://aws.amazon.com/tw/panorama/appliance/) for more information about your device.

3. **PLEASE READ THE [README](README.md) INCLUDE WITH THIS BEFORE YOU START USING THIS NOTEBOOK**

# Set up

Import libraries for use with this notebook environment, you do not need these libraries when you write your application code.

In [ ]:
import boto3

## Notebook parameters
Global constants that help the notebook create Panorama resources on your behalf.

In [ ]:
# application name
app_name = 'tf2_4_trt_app'

## package names and node names
code_package_name = 'tf2_4_trt_app'
camera_node_name = 'abstract_rtsp_media_source'

# AWS account ID
account_id = boto3.client("sts").get_caller_identity()["Account"]

## Set up application

Every application uses the creator's AWS Account ID as the prefix to uniquely identifies the application resources. Running `panorama-cli import-application` replaces the generic account Id with your account Id.

In [ ]:
!cd ./tf2_4_trt_app && panorama-cli import-application

## Download SSD Mobilenet TF-TRT Model (One Time Download)

In [ ]:
# remember to install wget beforehand
!cd ./tf2_4_trt_app/packages/{account_id}-tf2_4_trt_app-1.0 && bash download_model.sh

### Build Docker image from Dockerfile

Open a terminal, go to ./TF37_opengpu/dependencies/docker and run

```sudo docker build -t  tf37:latest . ```

### Build app with container

In [ ]:
container_asset_name = 'tf24_trt_app'

In [ ]:
%%capture captured_output

# Building container image.This process takes time (5min ~ 10min)
# FIXME : without %%capture, browser tab crashes because of too much output from the command.

!cd ./tf2_4_trt_app && panorama-cli build \
    --container-asset-name {container_asset_name} \
    --package-path packages/{account_id}-{code_package_name}-1.0 --local-image

In [ ]:
stdout_lines = captured_output.stdout.splitlines()
stderr_lines = captured_output.stderr.splitlines()
print("     :")
print("     :")
for line in stdout_lines[-30:] + stderr_lines[-30:]:
    print(line)

### Special flags in package.json

* Step 1 : Before you deploy the application, open TF37_opengpu/onnx_37_app/packages/(account-id)-tf2_4_trt_app-1.0/package.json
* Step 2 : Add the following flags to the package.json

```
"requirements": 
            [{
                    "type" : "hardware_access",
                    "inferenceAccelerators": [ 
                        {
                            "deviceType": "nvhost_gpu",
                            "sharedResourcePolicy": {
                                "policy" : "allow_all"
                            }
                        }
                    ]
            }]
```

The assets should look something like this

```
"assets": [
    {
        "name": "tf2_4_trt_app",
        "implementations": [
            {
                "type": "container",
                "assetUri": "9a49a98784f4571adacc417f00942dac7ef2e34686eef21dca9fcb7f4b7ffd70.tar.gz",
                "descriptorUri": "4bab130ec48eea84e072d9fe813b947e9d9610b2924099036b0165026a91d306.json",
                "requirements": 
                [{
                    "type" : "hardware_access",
                    "inferenceAccelerators": [ 
                        {
                            "deviceType": "nvhost_gpu",
                            "sharedResourcePolicy": {
                                "policy" : "allow_all"
                            }
                        }
                    ]
                }]
            }
        ]
    }
],
```

### Upload application to Panorama for deploying to devices

In [ ]:
# This step takes some time, depending on your network environment.
!cd ./tf2_4_trt_app && pwd && panorama-cli package-application

### Ready for deploying to a device

Congrats! Your app is now ready to deploy to a device. Next, you can continue in this notebook to deploy the app programmatically or you can go to the Panorama console and deploying using the AWS Console. The console makes it easier to select camera streams and select the devices you want to deploy to. Programmatic deployment is faster to complete and easier to automate.

# Deploy app to device

* Step 1: Copy the contents of./graphs/tf2_4_trt_app/graph.json
* Step 2 : Go to the AWS Panorama Console, click Deploy Application
* Step 3 : Paste the contents you just copied
* Step 4 : Select the device
* Step 5: Select atleast 2 cameras
* Step 6: Configure the batch size with desired batch size. 8 for Jetson Xavier AGX and 6 for Jetson NX. For more info about your Panorama spec, please refer to this [link](https://aws.amazon.com/tw/panorama/appliance/)
* Step 7 : Deploy

The deployment should take about 30 minutes